In [ ]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
np.set_printoptions(precision=5, suppress=True, linewidth=100)
plt.rcParams['figure.dpi'] = 150
from scipy.io import savemat
from tenpy.tools.params import get_parameter
from matplotlib import colors

In [ ]:
import tenpy
import tenpy.linalg.np_conserved as npc
from tenpy.algorithms import dmrg
from tenpy.networks.mps import MPS
from tenpy.models.tf_ising import TFIChain
tenpy.tools.misc.setup_logging(to_stdout="INFO")
import pickle

In [ ]:
from tenpy.networks.mps import MPS
from tenpy.models.tf_ising import TFIChain
from tenpy.algorithms import dmrg

from tenpy.networks.site import SpinSite, SpinHalfSite, SpinHalfFermionSite
from tenpy.models.lattice import Triangular, Square
from tenpy.models.model import CouplingModel, NearestNeighborModel, MPOModel

In [ ]:
''' Fermi-Hubbard model '''

class Hubbard_chiral(CouplingModel, MPOModel):
    def __init__(self, model_param):
        
        ''' system size '''
        Lx = model_param["Lx"]
        Ly = model_param["Ly"]
        
        ''' coupling constants'''
        t = model_param["t"]
        U = model_param["U"]
        phi = model_param["phase"]
        
        
        ''' boundary conditions'''
        bc_MPS = model_param["bc_MPS"]
        bc_y = model_param["bc_y"]
        bc_x = model_param["bc_x"]
        
        
        ''' site with particle + U(1) symmetry conservation'''
        site = SpinHalfFermionSite(cons_N = 'N', cons_Sz = 'Sz')
        
        ''' define triangular lattice'''
        lat = Triangular(Lx, Ly, site, bc=[bc_x, bc_y], bc_MPS=bc_MPS)   
        

        CouplingModel.__init__(self, lat)
        
        ''' on-site interactions'''
        for u in range(len(self.lat.unit_cell)):
            self.add_onsite(U, u, 'NuNd')
            
        ''' nearest-neighbors hopping'''
        for u1, u2, dx in self.lat.pairs['nearest_neighbors']:
            self.add_coupling(- t * np.exp(1j * phi), u1, 'Cdu', u2, 'Cu', dx, plus_hc=True)
            self.add_coupling(- t * np.exp(1j * phi), u1, 'Cdd', u2, 'Cd', dx, plus_hc=True)

            
        MPOModel.__init__(self, lat, self.calc_H_MPO())

In [ ]:
dmrg_params = {"trunc_params": {"chi_max": 100, "svd_min": 1.e-10}, "mixer": True, "max_sweeps": 500}

In [ ]:
model_param = {"Lattice" : Square,
              "Ly" : 6,
              "Lx" : 2,
              "t" : 1.0,
              "U" : 3.0,
              "phase" : np.pi/2,
              "bc_MPS" : "infinite",
              "bc_y" : 'periodic',
              "bc_x" : 'periodic'}

In [ ]:
M = Hubbard_chiral(model_param)
sites = M.lat.mps_sites()
psi = MPS.from_product_state(sites,["full","empty","empty"] * (M.lat.N_sites//3),"infinite")

In [ ]:
info = dmrg.run(psi, M, dmrg_params)
energy = info['E']
delta_E_energy[i] = abs(info["sweep_statistics"]['E'][-1] - info["sweep_statistics"]['E'][-2])

In [ ]:
data = {'psi': psi, 'dmrg_params': dmrg_params, 'Energy': energy, 'Delta Energy': delta_E_energy, 'model_params': model_param}

In [ ]:
with open('results_hubbard.pkl', 'wb') as f:
    pickle.dump(data, f)